In [0]:
Select * from dea_course_catalog_gizmobox.bronze.v_customers

create view to store all records where customer id is not null 

In [0]:
CREATE OR REPLACE TEMPORARY VIEW V_customers_distinct AS
SELECT distinct
  *
from
  dea_course_catalog_gizmobox.bronze.v_customers
WHERE
  customer_id IS NOT NULL
;
select * from V_customers_distinct

In [0]:
  select
    customer_id,
    max(created_timestamp) as max_created_timestamp
  from
    V_customers_distinct
  group by
    customer_id

Create cte 1 to group all records where timestamp is max and customer id is distinct and then join with main bronze table to eliminate outliers


In [0]:
WITH CTE_t1 AS (
  select
    customer_id,
    max(created_timestamp) as max_created_timestamp
  from
    V_customers_distinct
  group by
    customer_id
)
SELECT
  CAST(c.created_timestamp as TIMESTAMP) as created_timestamp,
  c.customer_id,
  c.customer_name,
  CAST(c.date_of_birth AS DATE) as date_of_birth,
  c.email,
  CAST(c.member_since AS DATE) as member_since,
  c.telephone
FROM
  V_customers_distinct c
    inner join cte_t1 t1
      on c.customer_id = t1.customer_id
      and c.created_timestamp = t1.max_created_timestamp

In [0]:
DROP TABLE IF EXISTS dea_course_catalog_gizmobox.silver.customers;

CREATE TABLE IF NOT EXISTS dea_course_catalog_gizmobox.silver.customers AS
WITH CTE_t1 AS (
  select
    customer_id,
    max(created_timestamp) as max_created_timestamp
  from
    V_customers_distinct
  group by
    customer_id
)
SELECT
  CAST(c.created_timestamp as TIMESTAMP) as created_timestamp,
  c.customer_id,
  c.customer_name,
  CAST(c.date_of_birth AS DATE) as date_of_birth,
  c.email,
  CAST(c.member_since AS DATE) as member_since,
  c.telephone
FROM
  V_customers_distinct c
    inner join cte_t1 t1
      on c.customer_id = t1.customer_id
      and c.created_timestamp = t1.max_created_timestamp

In [0]:
select * from dea_course_catalog_gizmobox.silver.customers

In [0]:
DESCRIBE EXTENDED dea_course_catalog_gizmobox.silver.customers